In [1]:
def infoGain(P,N):
    import math 
    return -P / (P + N) * math.log2(P / (P + N)) - N /(P + N) * math.log2(N / (P + N))

In [2]:
def insertNode(tree,addTo,Node):
    for k,v in tree.items():
        if isinstance(v,dict):
            tree[k] = insertNode(v,addTo,Node)
    if addTo in tree:
        if isinstance(tree[addTo],dict):
            tree[addTo][Node] = 'None'
        else:
            tree[addTo] = {Node:'None'}
    return tree

In [3]:
def insertConcept(tree,addTo,Node):
    for k,v in tree.items():
        if isinstance(v,dict):
            tree[k] = insertConcept(v,addTo,Node)
    if addTo in tree:
        tree[addTo] = Node
    return tree

In [4]:
def getNextNode(data,AttributeList,concept,conceptVals,tree,addTo):
    Total = data.shape[0]
    if Total == 0:
        return tree
    countC = {}
    for cVal in conceptVals:
        dataCC = data[data[concept] == cVal]
        countC[cVal] = dataCC.shape[0]
    if countC[conceptVals[0]] == 0:
        tree = insertConcept(tree,addTo,conceptVals[1])
        return tree
    if countC[conceptVals[1]] == 0:
        tree = insertConcept(tree,addTo,conceptVals[0])
        return tree
    ClassEntropy = infoGain(countC[conceptVals[0]],countC[conceptVals[1]])
    Attr = {}
    for a in AttributeList:
        Attr[a] = list(set(data[a]))
    AttrCount = {}
    EntropyAttr = {}
    for att in Attr:
        for vals in Attr[att]:
            for c in conceptVals:
                iData = data[data[att] == vals]
                dataAtt = iData[iData[concept] == c]
                AttrCount[c] = dataAtt.shape[0]
            TotalInfo = AttrCount[conceptVals[0]] + AttrCount[conceptVals[1]]
            if AttrCount[conceptVals[0]] == 0 or AttrCount[conceptVals[1]] == 0:
                InfoGain = 0
            else:
                InfoGain = infoGain(AttrCount[conceptVals[0]],AttrCount[conceptVals[1]])
            if att not in EntropyAttr:
                EntropyAttr[att] = (TotalInfo/Total) * InfoGain
            else:
                EntropyAttr[att] = EntropyAttr[att] + (TotalInfo/Total) * InfoGain
    Gain = {}
    for g in EntropyAttr:
        Gain[g] = ClassEntropy - EntropyAttr[g]
    Node = max(Gain,key=Gain.get)
    tree = insertNode(tree,addTo,Node)
    for nD in Attr[Node]:
        tree = insertNode(tree,Node,nD)
        newData = data[data[Node] == nD].drop(Node,axis=1)
        AttributeList = list(newData)[:-1]
        tree = getNextNode(newData,AttributeList,concept,conceptVals,tree,nD)
    return tree

In [5]:
import pandas as pd
def main():
    data = pd.read_csv('Datasets/PlayTennis.csv')
    AttributeList = list(data)[:-1]
    concept = str(list(data)[-1])
    conceptVals = list(set(data[concept]))
    tree = getNextNode(data,AttributeList,concept,conceptVals,{'root':'None'},'root')
    return tree

In [6]:
tree = main()['root']

In [7]:
df = pd.read_csv('Datasets/PlayTennis_test1.csv')
def test(tree,d):
    for k in tree:
        for v in tree[k]:
            if(d[k] == v and isinstance(tree[k][v],dict)):
                test(tree[k][v],d)
            elif(d[k] == v):
                print("Classification:" + tree[k][v])

In [8]:
print(tree)

{'Outlook': {'Overcast': 'Yes', 'Sunny': {'Humidity': {'High': 'No', 'Normal': 'Yes'}}, 'Rain': {'Wind': {'Strong': 'No', 'Weak': 'Yes'}}}}


In [9]:
test(tree,df.loc[0, :])

Classification:No
